# Part 5: Advanced - Few-Shot Learning으로 Review 품질 향상

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. PeerRead 데이터셋 활용법 이해
2. Few-shot learning으로 리뷰 품질 개선
3. 기본 AgentReview vs Few-shot enhanced 비교

⚠️ **선택 노트북**: Part 4 완료 후 시간이 있을 때 실습하세요

---
## Setup

In [ ]:
# Cell 1: 환경 설정 (Colab/로컬 자동 감지)
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경에서 실행 중")
except ImportError:
    # 로컬 환경
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: 패키지 설치
# Colab: 자동 설치
# 로컬: requirements.txt로 미리 설치 필요 (pip install -e .)

if IN_COLAB:
    print("📦 패키지 설치 중...")
    !pip install datasets google-generativeai python-dotenv -q
    print("✅ 설치 완료!")
else:
    print("✅ 로컬 환경: pyproject.toml로 설치된 패키지 사용")
    print("   (미설치 시: uv pip install -e . 또는 pip install -e .)")

In [ ]:
# Cell 3: API Key 로딩
import os
from pathlib import Path

# Colab 환경 체크
if IN_COLAB:
    # Colab userdata에서 키 불러오기
    try:
        from google.colab import userdata
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
        print("✅ Colab Secrets에서 API Key 로딩 완료")
    except:
        print("⚠️ Colab Secrets에 GEMINI_API_KEY를 추가하세요")
        print("   좌측 🔑 아이콘 > Add new secret")
        GEMINI_API_KEY = None
else:
    # 로컬 환경: dotenv 사용
    try:
        from dotenv import load_dotenv
        load_dotenv()
        GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
        
        if GEMINI_API_KEY:
            print("✅ .env 파일에서 API Key 로딩 완료")
        else:
            print("⚠️ .env 파일에 GEMINI_API_KEY를 추가하세요")
    except ImportError:
        print("⚠️ python-dotenv가 설치되지 않았습니다")
        GEMINI_API_KEY = None

# API 사용 가능 여부 확인
if GEMINI_API_KEY:
    print(f"🔑 API Key: {GEMINI_API_KEY[:10]}...")
else:
    print("❌ API Key를 설정하지 않으면 LLM 기능이 동작하지 않습니다")

---
## 1. 파악하기: PeerRead 데이터셋이란?

### 데이터셋 개요

**PeerRead**는 최초의 공개 peer review 데이터셋입니다 (NAACL 2018).

| 항목 | 내용 |
|------|------|
| **규모** | 14K+ 논문, 10K+ expert reviews |
| **출처** | ACL, NeurIPS, ICLR (top-tier venues) |
| **포함 정보** | 논문 초안, 리뷰 코멘트, accept/reject 결정 |
| **접근성** | Hugging Face에서 즉시 사용 가능 |

### Why PeerRead?

Review-CoT 데이터셋(142k reviews)은 공개되지 않았지만, PeerRead는:
- ✅ Hugging Face에서 3줄 코드로 로딩
- ✅ 실제 top-tier conference reviews
- ✅ Accept/reject decisions 포함
- ✅ Few-shot learning에 충분한 품질

**GitHub**: [allenai/PeerRead](https://github.com/allenai/PeerRead)  
**Paper**: [arXiv:1804.09635](https://arxiv.org/abs/1804.09635)

---
## 2. 써보기: PeerRead 데이터셋 로딩

In [ ]:
# Cell 4: 데이터셋 로딩
from datasets import load_dataset

print("📦 PeerRead 데이터셋 로딩 중...")
print("   (처음 실행 시 다운로드에 1-2분 소요됩니다)\n")

# PeerRead 전체 데이터셋 로딩
peerread = load_dataset("allenai/peer_read", "full", split="train")

print(f"✅ 로딩 완료!")
print(f"📊 총 {len(peerread)}개 리뷰")
print(f"\n데이터셋 컬럼: {peerread.column_names}")

In [ ]:
# Cell 5: 데이터셋 탐색
import pandas as pd

# Accept/Reject 분포
decisions = pd.Series([r['RECOMMENDATION'] for r in peerread if 'RECOMMENDATION' in r])
print("📊 Accept/Reject 분포:")
print(decisions.value_counts())
print()

# 리뷰 길이 분포
review_lengths = [len(r.get('COMMENTS', '')) for r in peerread]
print(f"📏 리뷰 길이 통계:")
print(f"   평균: {pd.Series(review_lengths).mean():.0f} 글자")
print(f"   중앙값: {pd.Series(review_lengths).median():.0f} 글자")
print(f"   최대: {max(review_lengths)} 글자")

In [ ]:
# Cell 6: 샘플 리뷰 확인
# Accept된 논문 중 첫 번째 리뷰 확인

sample = None
for review in peerread:
    if review.get('RECOMMENDATION') == 'accept' and review.get('COMMENTS'):
        sample = review
        break

if sample:
    print("📄 샘플 리뷰 (Accept)\n")
    print(f"논문 제목: {sample.get('TITLE', 'N/A')}")
    print(f"결정: {sample.get('RECOMMENDATION', 'N/A')}")
    print(f"\n리뷰 코멘트 (처음 500자):\n")
    print(sample.get('COMMENTS', 'N/A')[:500])
    print("\n...")
else:
    print("⚠️ Accept 리뷰 샘플을 찾을 수 없습니다")

---
## 3. 바꿔보기: Few-Shot Learning 구현

### Few-Shot Learning이란?

LLM에게 **몇 개의 예시를 보여준 후** 같은 스타일로 생성하도록 하는 기법입니다.

```
프롬프트:
"다음은 고품질 리뷰의 예시입니다:

Example 1: [accept된 논문의 리뷰 1]
Example 2: [accept된 논문의 리뷰 2]
...

이제 이 논문을 같은 스타일로 리뷰하세요:
[본인 논문]
"
```

### 효과
- ✅ 리뷰 형식과 톤을 학습
- ✅ 구체적인 피드백 증가
- ✅ Top-tier conference 수준의 깊이

In [ ]:
# Cell 7: 고품질 리뷰 목록 표시

# Accept된 고품질 리뷰 필터링 (최소 길이 500자)
high_quality_candidates = [
    (i, r) for i, r in enumerate(peerread)
    if r.get('RECOMMENDATION') == 'accept'
    and len(r.get('COMMENTS', '')) >= 500
]

print(f"✅ Accept된 고품질 리뷰: {len(high_quality_candidates)}개\n")
print("📋 리뷰 목록 (처음 20개):")
print("=" * 80)

for idx, (original_idx, review) in enumerate(high_quality_candidates[:20]):
    title = review.get('TITLE', 'N/A')[:60]
    length = len(review['COMMENTS'])
    print(f"[{idx:2d}] {title}... ({length:,}자)")

print("\n💡 본인 연구 분야와 유사한 리뷰를 5개 골라주세요!")
print("   (다음 셀에서 인덱스 번호로 선택)")

In [ ]:
# DIY Cell 8: 원하는 리뷰 선택하기

# 위의 목록에서 본인 분야와 유사한 리뷰의 인덱스를 입력하세요
selected_indices = [0, 3, 7, 12, 15]  # <- 이 숫자들을 수정하세요!

# 선택된 리뷰 추출
few_shot_examples = [high_quality_candidates[i][1] for i in selected_indices if i < len(high_quality_candidates)]

print(f"✅ {len(few_shot_examples)}개 리뷰 선택 완료\n")
print("선택된 리뷰:")
for i, ex in enumerate(few_shot_examples, 1):
    print(f"{i}. {ex.get('TITLE', 'N/A')[:60]}... (길이: {len(ex['COMMENTS'])} 글자)")

In [ ]:
# Cell 9: Few-Shot 프롬프트 구성

def build_few_shot_prompt(examples, paper_abstract):
    """Few-shot learning을 위한 프롬프트 생성"""
    
    # Examples 섹션
    examples_text = "Here are examples of high-quality peer reviews from top-tier conferences (ACL, NeurIPS, ICLR):\n\n"
    
    for i, ex in enumerate(examples, 1):
        review = ex['COMMENTS'][:800]  # 너무 길면 잘라내기
        examples_text += f"=== Example {i} ===\n{review}\n\n"
    
    # Task 섹션
    task_text = f"""Now, write a peer review for the following paper in the same style and depth:

Paper Abstract:
{paper_abstract}

Provide a thorough review covering:
1. Summary of the paper's main contributions
2. Strengths
3. Weaknesses
4. Questions for the authors
5. Overall recommendation
"""
    
    return examples_text + task_text

# 테스트용 논문 초록
test_abstract = """
We propose a novel approach to depression detection using multimodal analysis of 
clinical interviews. Our method combines audio features (prosody, speech rate) with 
text embeddings from transformer models to predict PHQ-8 scores. Experiments on the 
DAIC-WOZ dataset show 15% improvement over text-only baselines.
"""

# 선택된 리뷰로 프롬프트 생성
few_shot_prompt = build_few_shot_prompt(few_shot_examples, test_abstract)

print("✅ Few-shot 프롬프트 생성 완료")
print(f"   총 길이: {len(few_shot_prompt)} 글자")
print("\n프롬프트 미리보기 (처음 500자):")
print(few_shot_prompt[:500])
print("\n...")

# Cell 10: LLM으로 Few-Shot Review 생성
import google.generativeai as genai

if not GEMINI_API_KEY:
    print("❌ API Key가 설정되지 않았습니다. Cell 3을 먼저 실행하세요.")
else:
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
    
    print("🤖 Few-shot learning으로 리뷰 생성 중...\n")
    
    response = model.generate_content(few_shot_prompt)
    few_shot_review = response.text
    
    print("=== Few-Shot Enhanced Review ===\n")
    print(few_shot_review)

In [ ]:
# Cell 10: 기본 프롬프트로 리뷰 생성 (비교용)

basic_prompt = f"""You are a peer reviewer for a top-tier conference. 
Write a review for the following paper:

{test_abstract}

Provide:
1. Summary
2. Strengths
3. Weaknesses
4. Questions
5. Recommendation
"""

if GEMINI_API_KEY:
    print("🤖 기본 프롬프트로 리뷰 생성 중...\n")
    
    response_basic = model.generate_content(basic_prompt)
    basic_review = response_basic.text
    
    print("=== Basic Review (Without Few-Shot) ===\n")
    print(basic_review)

In [ ]:
# Cell 11: 기본 프롬프트로 리뷰 생성 (비교용)

basic_prompt = f"""You are a peer reviewer for a top-tier conference. 
Write a review for the following paper:

{test_abstract}

Provide:
1. Summary
2. Strengths
3. Weaknesses
4. Questions
5. Recommendation
"""

if GEMINI_API_KEY:
    print("🤖 기본 프롬프트로 리뷰 생성 중...\n")
    
    response_basic = model.generate_content(basic_prompt)
    basic_review = response_basic.text
    
    print("=== Basic Review (Without Few-Shot) ===\n")
    print(basic_review)

# Cell 12: 차이점 분석

print("📊 기본 vs Few-Shot 비교\n")

if GEMINI_API_KEY:
    print(f"기본 리뷰 길이: {len(basic_review)} 글자")
    print(f"Few-shot 리뷰 길이: {len(few_shot_review)} 글자")
    print()
    
    # 구조화 정도 (질문 개수로 간단히 측정)
    basic_questions = basic_review.count('?')
    fewshot_questions = few_shot_review.count('?')
    
    print(f"질문 개수 (구체성):")
    print(f"  - 기본: {basic_questions}개")
    print(f"  - Few-shot: {fewshot_questions}개")
    print()
    
    print("💡 관찰 포인트:")
    print("  - Few-shot 리뷰가 더 구체적인가?")
    print("  - PeerRead 예시의 톤과 형식을 따르는가?")
    print("  - 기술적 깊이가 더 있는가?")
else:
    print("⚠️ API Key 없이는 비교 불가")

In [ ]:
# DIY Cell 12: 본인 논문 초록 입력

my_paper_abstract = """
[여기에 본인 논문 초록을 입력하세요]

예:
This study investigates...
"""

# Few-shot 프롬프트 생성
my_few_shot_prompt = build_few_shot_prompt(few_shot_examples, my_paper_abstract)

print(f"✅ 프롬프트 생성 완료 (길이: {len(my_few_shot_prompt)} 글자)")

In [ ]:
# DIY Cell 13: 본인 논문 초록 입력

my_paper_abstract = """
[여기에 본인 논문 초록을 입력하세요]

예:
This study investigates...
"""

# Few-shot 프롬프트 생성 (Cell 8에서 선택한 리뷰 사용)
my_few_shot_prompt = build_few_shot_prompt(few_shot_examples, my_paper_abstract)

print(f"✅ 프롬프트 생성 완료 (길이: {len(my_few_shot_prompt)} 글자)")

# DIY Cell 14: 본인 논문 리뷰 생성

if not GEMINI_API_KEY:
    print("❌ API Key가 필요합니다")
elif "[여기에" in my_paper_abstract:
    print("⚠️ 위 셀에서 본인의 논문 초록을 먼저 입력하세요")
else:
    print("🤖 본인 논문 리뷰 생성 중...\n")
    
    response_my = model.generate_content(my_few_shot_prompt)
    my_review = response_my.text
    
    print("=== Your Paper Review (Few-Shot Enhanced) ===\n")
    print(my_review)

In [ ]:
# Cell 14: 분야별 필터링

def filter_by_field(dataset, keywords):
    """특정 분야의 리뷰만 필터링"""
    filtered = [
        r for r in dataset
        if any(kw.lower() in r.get('TITLE', '').lower() for kw in keywords)
    ]
    return filtered

# 예: 심리학/인지과학 관련 리뷰만
psych_keywords = ['psychology', 'cognitive', 'mental', 'depression', 'emotion']
psych_reviews = filter_by_field(peerread, psych_keywords)

print(f"심리학 관련 리뷰: {len(psych_reviews)}개")

if psych_reviews:
    print("\n샘플 논문 제목:")
    for r in psych_reviews[:5]:
        print(f"- {r.get('TITLE', 'N/A')[:60]}...")

In [ ]:
# Cell 15: 분야별 필터링

def filter_by_field(dataset, keywords):
    """특정 분야의 리뷰만 필터링"""
    filtered = [
        r for r in dataset
        if any(kw.lower() in r.get('TITLE', '').lower() for kw in keywords)
    ]
    return filtered

# 예: 심리학/인지과학 관련 리뷰만
psych_keywords = ['psychology', 'cognitive', 'mental', 'depression', 'emotion']
psych_reviews = filter_by_field(peerread, psych_keywords)

print(f"심리학 관련 리뷰: {len(psych_reviews)}개")

if psych_reviews:
    print("\n샘플 논문 제목:")
    for i, r in enumerate(psych_reviews[:10]):
        print(f"[{i}] {r.get('TITLE', 'N/A')[:60]}...")
    
    print("\n💡 분야별로 필터링한 후 Cell 7-8처럼 선택할 수도 있습니다!")

# DIY Cell 16: 본인 분야 키워드로 커스텀

my_keywords = ['your', 'field', 'keywords']  # <- 수정!

my_field_reviews = filter_by_field(peerread, my_keywords)
print(f"본인 분야 리뷰: {len(my_field_reviews)}개")

if my_field_reviews:
    print("\n샘플 논문 제목:")
    for i, r in enumerate(my_field_reviews[:10]):
        print(f"[{i}] {r.get('TITLE', 'N/A')[:60]}...")
    
    # 이 리뷰들로 few-shot 재구성
    if len(my_field_reviews) >= 5:
        print(f"\n✅ 충분한 리뷰가 있습니다!")
        print("   이 목록에서 5개를 선택해 few-shot examples로 사용할 수 있습니다.")

---
## 참고 자료

### PeerRead
- **GitHub**: https://github.com/allenai/PeerRead
- **Paper**: https://arxiv.org/abs/1804.09635
- **Hugging Face**: https://huggingface.co/datasets/allenai/peer_read

### OpenReview
- **Platform**: https://openreview.net/
- NeurIPS, ICLR, ICML 등의 리뷰 공개

### Few-Shot Learning
- OpenAI: [Best practices for prompt engineering](https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api)
- Anthropic: [Prompt engineering guide](https://docs.anthropic.com/claude/docs/prompt-engineering)

---

**완료! 🎉**

Part 5에서 배운 내용:
- PeerRead 데이터셋 활용
- Few-shot learning 구현
- 리뷰 품질 비교 및 분석
- 분야별 커스텀 방법